

* **Retrieval Augmented Generation (RAG)**

    RAG is a technique in Natural Language Processing (NLP) that improves text generation. Here's how it works:

    1. **Retrieval:**  A query triggers a search through a knowledge base to find relevant information.
    2. **Generation:**  A language model uses the retrieved information to create a factual and fluent response.

* **Benefits of RAG**

    * **Limits errors:** Helps prevent inaccurate or misleading outputs from the language model.  
    * **Provides better responses:**  Offers more informative and relevant responses to user queries.

* **Using `.env` Files**

    * **Why they're used:**  `.env` files store sensitive project information like API keys for accessing knowledge bases or other external services. 
    * **Security:** Keeps this sensitive data out of your main code, improving security.

**Important Note**

This is just the initial setup. A full RAG implementation would include:

* **A knowledge base:**  The source of information to retrieve.
* **Retrieval model:**  The component that searches for relevant information.
* **Generative model:**  The language model that produces text responses.
* **Integration code:**  Logic to tie together the retrieval and generation steps.



## Steps

## Building a RAG Model with `llama`

The `llama_index` library simplifies the process of creating and working with vector indexes, which are crucial for building Retrieval-Augmented Generation (RAG) models. The main steps involved are:

1. **Data Ingestion**
  - Load data from various sources (directories, PDFs, CSVs, etc.) using provided readers (`SimpleDirectoryReader`, `PDFReader`, `CSVReader`, etc.).

2. **Document Chunking**
  - Split large documents into smaller chunks for efficient indexing.
  - Automatic chunking based on configurable parameters (max chunk size, sentence boundaries, etc.).

3. **Vector Index Creation**
  - Create a vector index that maps textual data to dense vector representations.
  - Choose from different types of vector stores (`SimpleVectorStore`, `GPTVectorStoreIndex`, etc.).

4. **Index Persistence**
  - Persist the created vector index to disk using `StorageContext`.
  - Avoid recreating the index from scratch every time.

5. **Query Processing**
  - Use `QueryEngine` to process queries.
  - Retrieve relevant chunks from the vector index based on semantic similarity.
  - Pass retrieved chunks to a language model for response generation.

6. **Response Generation**
  - Integrate with various language models (GPT-3, BERT, etc.).
  - Generate a response based on the query and retrieved context chunks.


In [1]:
## Retrieval augmented generation

import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")


**SimpleDirectoryReader: Easy Data Loading for LlamaIndex**

SimpleDirectoryReader is a straightforward way to load data from your local files into LlamaIndex. It's ideal for getting started quickly. For production, consider the specialized Readers on LlamaHub.

**Supported File Types**

* Plain text (.txt)
* .csv, .docx, .epub, .hwp, .ipynb 
* Image formats (.jpeg, .jpg, .png)
* .mbox, .md, .pdf, 
* Audio/video (.mp3, .mp4) 
* PowerPoint (.ppt, .pptm, .pptx)

**Note:** Use JSON Loader for JSON files.

**Full documentation:** https://docs.llamaindex.ai/en/stable/module_guides/loading/simpledirectoryreader/ 


**Usage**

```python
from llama_index.core import SimpleDirectoryReader

reader = SimpleDirectoryReader(input_dir="path/to/directory")
documents = reader.load_data()


In [13]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader

# load_data has the following optional inputs:
documents = SimpleDirectoryReader("data").load_data()

In [14]:
documents

[Document(id_='d2ce2155-cee3-4cdc-9925-807abecd6442', embedding=None, metadata={'page_label': '1', 'file_name': 'paper.pdf', 'file_path': 'data\\paper.pdf', 'file_type': 'application/pdf', 'file_size': 2025992, 'creation_date': '2024-04-29', 'last_modified_date': '2024-04-19', 'last_accessed_date': '2024-04-29'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='Glaucoma\nDoes Glaucoma Alter Eye Movements When Viewing\nImages of Natural Scenes? A Between-Eye Study\nDaniel S. Asfaw, Pete R. Jones, Vera M. M¨ onter, Nicholas D. Smith, and David P . Crabb\nDivision of Optometry and Visual Science, School of Health Science, City, University of London, London, United Kingdom\nCorrespondence: David P . Crabb,\nDivision of Optometry and Visual\nScience, School of

**VectorStoreIndex.from_documents(...)**

* **Key Function:** Creates your core searchable index.

* **Input:**
    * `documents`: List of `Document` objects generated in a previous step.
    * `show_progress=True`:  Enables a progress bar during index creation.

* **Output:**
    * A `VectorStoreIndex` object: A specialized database for storing and querying language model-generated text embeddings.

**index**

* **Variable:**  Stores the created `VectorStoreIndex`. You'll use this for:
    * Adding more documents.
    * Searching the index using natural language.


In [15]:
index = VectorStoreIndex.from_documents(documents,show_progress=True)
index

Parsing nodes:   0%|          | 0/10 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/16 [00:00<?, ?it/s]

In [16]:
from llama_index.retrievers import VectorIndexRetriever
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.indices.postprocessor import SimilarityPostprocessor

retriever=VectorIndexRetriever(index=index,similarity_top_k=4)
postprocessor=SimilarityPostprocessor(similarity_cutoff=0.80)

query_engine=RetrieverQueryEngine(retriever=retriever,
                                  node_postprocessors=[postprocessor])


In [17]:
response=query_engine.query("What is glaoucoma?")


In [18]:

from llama_index.response.pprint_utils import pprint_response
pprint_response(response,show_source=True)
print(response)

Final Response: Glaucoma is a condition that affects the eyes and can
lead to altered eye movements. It is associated with changes in the
optic nerve and can result in visual field loss.
______________________________________________________________________
Source Node 1/2
Node ID: f756b226-9ea4-45d7-b9a1-b88abfeb7bdd
Similarity: 0.8167331642355051
Text: One possibility may be to measure a person’s natural eye
movements. Glaucoma patients have been shown to have altered eye
movements as compared to peers with normal vision when performing
everyday tasks, such as reading,3–5 visual search,6face
recognition,7watching video,8driving,9–12 and viewing images13(for a
review, see Kasneci et al.14). Fur...
______________________________________________________________________
Source Node 2/2
Node ID: 3d5f1cb5-e325-4f97-a85c-b5e7e7b4c59b
Similarity: 0.8084063992525515
Text: Glaucoma Does Glaucoma Alter Eye Movements When Viewing Images
of Natural Scenes? A Between-Eye Study Daniel S. Asfaw, Pet

In [19]:
prompt = "Can eye movments be used to detect gloucoma?"
response=query_engine.query(prompt)

from llama_index.response.pprint_utils import pprint_response
pprint_response(response,show_source=True)
print(response)

Final Response: Eye movements can potentially be used as biomarkers
for detecting glaucoma. Studies have shown that individuals with
glaucoma exhibit altered eye movements compared to those with normal
vision when performing various tasks. These changes in eye movements
have been linked to visual field loss in glaucoma patients.
Researchers have explored measuring natural eye movements as a
possible method to detect and monitor glaucoma, offering an
alternative or complementary approach to traditional visual field
assessments.
______________________________________________________________________
Source Node 1/4
Node ID: f756b226-9ea4-45d7-b9a1-b88abfeb7bdd
Similarity: 0.8765306052657766
Text: One possibility may be to measure a person’s natural eye
movements. Glaucoma patients have been shown to have altered eye
movements as compared to peers with normal vision when performing
everyday tasks, such as reading,3–5 visual search,6face
recognition,7watching video,8driving,9–12 and viewing